# Hello Hybrid Jobs with Cuda-Quantum

## Build your own container with cuda-quantum

CUDA-Quantum is a platform for hybrid quantum-classical computing. It offers a unified programming model designed for a hybrid setting for CPUs and GPUs. CUDA-Q contains support for programming in Python and in C++. 

In this section, a job container is built with CUDA-Quanum and other GPU related settings configured. The procedure for BYOC is presented in this page from [Braker Developer Guide](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-byoc.html). The required files for building a container with CUDA-Quantum is the "container" folder, including
- Dockerfile: Describes how the container is built.
- requirements.txt: Additional Python dependencies to include.
- braket_container.py: The start up script of a job container (optional).

In addition, there is a shell script "container_build_and_push.sh" automates the procedure of pulling, building and pushing container images. The shell script takes two paramerers:
- image-name
- region-name

The shell script is called with the following syntax
```
container/container_build_and_push.sh <image-name> <region-name>
```

In [27]:
! container/container_build_and_push.sh braket-cudaq-byoc-job us-west-2

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 937B done
#1 DONE 0.0s

#2 [internal] load metadata for 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-pytorch-jobs:latest
#2 ...

#3 [auth] sharing credentials for 292282985366.dkr.ecr.us-west-2.amazonaws.com
#3 DONE 0.0s

#2 [internal] load metadata for 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-pytorch-jobs:latest
#2 DONE 0.5s

#4 [internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [1/5] FROM 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-pytorch-jobs:latest@sha256:fe5f0c70d6b5cc587e24259c549aa1e221e0009885a065537028cbbd0193c06a
#5 DONE 0.0s

#6 [internal] load build context
#6 transferring context: 1

## Hybrid job with BYOC

In [28]:
import numpy as np

from braket.jobs import hybrid_job
from braket.jobs.config import InstanceConfig
from braket.jobs.environment_variables import get_job_device_arn

image_uri = "537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest"

## first local cudaq job

In [29]:
@hybrid_job(device='local:nvidia/qpp-cpu', image_uri=image_uri, local=True)
def hello_quantum(sagemaker_mpi_enabled=True):
    import cudaq
    
    device=get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print(cudaq.get_target())
    
    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)
    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])
    
    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print(measurement_probabilities)
    
    return measurement_probabilities

Skipping python version validation, make sure versions match between local environment and container.


In [30]:
hello_quantum()

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Pulling docker container image. This may take a while.


Login Succeeded
latest: Pulling from braket-cudaq-byoc-job
Digest: sha256:2218a197d02e04b9ade52f61b67c36ac94ccc9d1866b7fb2637e5e94117f8e4b
Status: Image is up to date for 537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest
537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest



What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview 537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest


Boto3 Version:  1.34.96
Beginning Setup
Checking for Additional Requirements
Additional Requirements Check Finished
Running Code As Process
Target qpp-cpu
	simulator=qpp
	platform=default
	description=QPP-based CPU-only backend target
	precision=fp64

{'11': 493, '00': 507}
Code Run Finished
7e4a32ae0a1904a26fc30759f6c974b0ec2958d3d39328afcbbcfa16ff0a9eac


## First remote cudaq job

In [31]:
@hybrid_job(device='local:nvidia/qpp-cpu', image_uri=image_uri)
def hello_quantum(sagemaker_mpi_enabled=True):
    import cudaq
    
    device=get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print(cudaq.get_target())
    
    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)
    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])
    
    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print(measurement_probabilities)
    
    return measurement_probabilities

job = hello_quantum()
job.arn

Skipping python version validation, make sure versions match between local environment and container.


'arn:aws:braket:us-west-2:537332306153:job/4bb306c4-c703-404b-8dba-6d25e58ff08b'

## write it differently with @cuda.kernel

In [32]:
def hello_quantum_deco_kernel(sagemaker_mpi_enabled=True):
    import cudaq
    
    device=get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print(cudaq.get_target())
    
    @cudaq.kernel
    def kernel():
        q0 = cudaq.qubit()
        q1 = cudaq.qubit()
        h(q0)
        cx(q0, q1)

    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print(measurement_probabilities)
    return measurement_probabilities

In [33]:
job_call = hybrid_job(device='local:nvidia/qpp-cpu', image_uri=image_uri, local=True)(
    hello_quantum_deco_kernel
)

job_call()

Skipping python version validation, make sure versions match between local environment and container.


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Pulling docker container image. This may take a while.


Login Succeeded
latest: Pulling from braket-cudaq-byoc-job
Digest: sha256:2218a197d02e04b9ade52f61b67c36ac94ccc9d1866b7fb2637e5e94117f8e4b
Status: Image is up to date for 537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest
537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest



What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview 537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest


Boto3 Version:  1.34.96
Beginning Setup
Checking for Additional Requirements
Additional Requirements Check Finished
Running Code As Process
Target qpp-cpu
	simulator=qpp
	platform=default
	description=QPP-based CPU-only backend target
	precision=fp64

Process Process-1:
RuntimeError: join is not a valid kernel to call.
Code Run Finished
Process exited with code: 1
26e8cf8e69627a634c89355e0058d20a506c4d01d8b8517cf30d6d0d025641a4


In [34]:
import autoqasm as aq

job_call = aq.hybrid_job(device='local:nvidia/qpp-cpu', image_uri=image_uri, local=True)(
    hello_quantum_deco_kernel
)

job_call()

Skipping python version validation, make sure versions match between local environment and container.


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Pulling docker container image. This may take a while.


Login Succeeded
latest: Pulling from braket-cudaq-byoc-job
Digest: sha256:2218a197d02e04b9ade52f61b67c36ac94ccc9d1866b7fb2637e5e94117f8e4b
Status: Image is up to date for 537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest
537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest



What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview 537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest


Boto3 Version:  1.34.96
Beginning Setup
Checking for Additional Requirements
Additional Requirements Check Finished
Running Code As Process
Target qpp-cpu
	simulator=qpp
	platform=default
	description=QPP-based CPU-only backend target
	precision=fp64

{'11': 502, '00': 498}
Code Run Finished
d00d373c3e78cec390c110c37c72b7344ec1fbb506db3dad9ebe047b8b2f5388


## result

In [11]:
from braket.aws import AwsQuantumJob

job_arn = "arn:aws:braket:us-west-2:537332306153:job/428a18d0-8700-4865-adb5-d23138b74500"
job = AwsQuantumJob(job_arn)

In [12]:
job.result()

{'11': 475, '00': 525}